In [1]:
import numpy as np
import pandas as pd

# Función para calcular la entropía de un conjunto de datos
def entropy(target_col):
    elements, counts = np.unique(target_col, return_counts=True)
    return np.sum([(-count / np.sum(counts)) * np.log2(count / np.sum(counts)) for count in counts])

# Función para calcular la ganancia de información de un atributo
def InfoGain(data, split_attribute_name, target_name):
    total_entropy = entropy(data[target_name])
    vals, counts = np.unique(data[split_attribute_name], return_counts=True)
    Weighted_Entropy = np.sum([(counts[i] / np.sum(counts)) * entropy(data.where(data[split_attribute_name] == vals[i]).dropna()[target_name]) for i in range(len(vals))])
    return total_entropy - Weighted_Entropy

# Función para predecir una clase para una nueva fila
def predict(row, tree):
    while isinstance(tree, dict):
        root_node = next(iter(tree))
        node_value = row.get(root_node, None)
        if node_value in tree[root_node]:
            tree = tree[root_node][node_value]
        else:
            # Se retorna la clase más común si el valor no está en el árbol
            return most_common_class(tree[root_node])
    return tree

# Se encuentra la clase más común en un nodo del árbol. 
def most_common_class(node):
    if isinstance(node, dict):
        # Se crea una lista plana de todos los valores no-dict del árbol
        leaf_values = []
        def get_leaf_values(subnode):
            if isinstance(subnode, dict):
                for value in subnode.values():
                    get_leaf_values(value)
            else:
                leaf_values.append(subnode)
        get_leaf_values(node)
        # Se devuelve el valor más común en la lista de valores hoja
        return max(set(leaf_values), key=leaf_values.count)
    return node


# Función principal para el algoritmo ID3
def ID3(data, originaldata, features, target_attribute_name, parent_node_class=None):
    if len(data) == 0:
        return parent_node_class
    elif len(np.unique(data[target_attribute_name])) <= 1:
        return data[target_attribute_name].iloc[0]
    elif len(features) == 0:
        return np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])]
    else:
        # Se calcula la ganancia de información para cada atributo
        item_values = [InfoGain(data, feature, target_attribute_name) for feature in features]
        # Se obtiene el atributo con la mayor ganancia de información
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        # Se construye el árbol
        tree = {best_feature: {}}
        # Se elimina el atributo con la mayor ganancia de información
        features = [i for i in features if i != best_feature]

        # Se construye un árbol por cada posible valor del atributo
        for value in np.unique(data[best_feature]):
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = ID3(sub_data, data, features, target_attribute_name, np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])])
            tree[best_feature][value] = subtree
        return tree


In [2]:
# Funciones para la validación cruzada
def cross_validation_split(data, folds):
    data_split = np.array_split(data, folds)
    return data_split

# Se define CV estratificado. Intenta mantener la proporción de datos de cada clase
def stratified_cross_validation_split(data, folds, target_attribute_name):
    data_split = []
    unique_classes = np.unique(data[target_attribute_name])
    for _ in range(folds):
        fold = pd.DataFrame()
        for cls in unique_classes:
            class_data = data[data[target_attribute_name] == cls]
            fold = pd.concat([fold, class_data.sample(frac=1/folds)])
        data_split.append(fold)
    return data_split

In [3]:
# Se define la validación cruzada
def cross_validate(data, folds, target_attribute_name, stratified=False):
    if stratified:
        splits = stratified_cross_validation_split(data, folds, target_attribute_name)
    else:
        splits = cross_validation_split(data, folds)
    accuracies = []

    # Se aplica un ciclo for para que realice todo el proceso de CV
    for fold in splits:
        train_set = pd.concat([df for df in splits if df is not fold])
        test_set = fold
        features = [col for col in data.columns if col != target_attribute_name]
        tree = ID3(train_set, train_set, features, target_attribute_name)
        predictions = test_set.apply(lambda row: predict(row, tree), axis=1)
        accuracy = (predictions == test_set[target_attribute_name]).mean()
        accuracies.append(accuracy)

    return accuracies

In [4]:
data = pd.read_csv("C:\\Users\\Carlo\\Desktop\\IA\\segundo semestre\\apredizaje automatico\\tarea\\base de datos discretizadas\\discretized_Yeast.csv")
data = data.drop('Sequence_Name', axis=1) 
accuracies_simple = cross_validate(data, 10, "class", stratified=False)
print("Precisión media (simple):", np.mean(accuracies_simple))
print("Desviación estándar de la precisión (simple):", np.std(accuracies_simple))
accuracies_stratified = cross_validate(data, 10, "class", stratified=True)
print("Precisión media (estratificada):", np.mean(accuracies_stratified))
print("Desviación estándar de la precisión (estratificada):", np.std(accuracies_stratified))

c:\Users\Carlo\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Precisión media (simple): 0.5450934155632143
Desviación estándar de la precisión (simple): 0.05214508850003015
Precisión media (estratificada): 0.5687074829931973
Desviación estándar de la precisión (estratificada): 0.02999375195244707
